In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.svm import SVC
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import confusion_matrix
from keras.layers import Dense, Activation, Dropout
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import random

from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import LeaveOneGroupOut
import joblib

In [2]:
df1 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Location/raw_location_features_studentlife.csv')
df1 = df1.rename(columns={'Date': 'date'})
df1 = df1.drop('Unnamed: 0', axis=1)
df2 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Stress/recreating_dailystress_features.csv')
df2 = df2.drop('Unnamed: 0', axis=1)

In [3]:
df = pd.merge(df1, df2, on=['uid', 'date'])
print(df.columns)

df = df.sort_values(by='date')

Index(['Unnamed: 0.1', 'Unnamed: 0_x', 'uid', 'date', 'number_places_morning',
       'number_places_afternoon', 'number_places_evening',
       'number_places_night', 'variance_time_spent_morning',
       'variance_time_spent_afternoon', 'variance_time_spent_evening',
       'variance_time_spent_night', 'time_spent_morning_firstplace',
       'time_spent_afternoon_firstplace', 'time_spent_evening_firstplace',
       'time_spent_night_firstplace', 'time_spent_morning_secondplace',
       'time_spent_afternoon_secondplace', 'time_spent_evening_secondplace',
       'time_spent_night_secondplace', 'time_spent_morning_thirdplace',
       'time_spent_afternoon_thirdplace', 'time_spent_evening_thirdplace',
       'time_spent_night_thirdplace', 'Unnamed: 0_y',
       'time_spent_morning_1stplace', 'time_spent_afternoon_1stplace',
       'time_spent_evening_1stplace', 'time_spent_night_1stplace',
       'time_spent_morning_2ndplace', 'time_spent_afternoon_2ndplace',
       'time_spent_evening_

In [4]:
df.isnull().sum()

Unnamed: 0.1              0
Unnamed: 0_x              0
uid                       0
date                      0
number_places_morning     0
                         ..
nothome_stay_morning      0
nothome_stay_afternoon    0
nothome_stay_evening      0
nothome_stay_night        0
stress_ratings            0
Length: 86, dtype: int64

In [5]:
df = df.dropna()
print(len(df))

1203


In [6]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0_x', 'uid', 'date', 'number_places_morning',
       'number_places_afternoon', 'number_places_evening',
       'number_places_night', 'variance_time_spent_morning',
       'variance_time_spent_afternoon', 'variance_time_spent_evening',
       'variance_time_spent_night', 'time_spent_morning_firstplace',
       'time_spent_afternoon_firstplace', 'time_spent_evening_firstplace',
       'time_spent_night_firstplace', 'time_spent_morning_secondplace',
       'time_spent_afternoon_secondplace', 'time_spent_evening_secondplace',
       'time_spent_night_secondplace', 'time_spent_morning_thirdplace',
       'time_spent_afternoon_thirdplace', 'time_spent_evening_thirdplace',
       'time_spent_night_thirdplace', 'Unnamed: 0_y',
       'time_spent_morning_1stplace', 'time_spent_afternoon_1stplace',
       'time_spent_evening_1stplace', 'time_spent_night_1stplace',
       'time_spent_morning_2ndplace', 'time_spent_afternoon_2ndplace',
       'time_spent_evening_

In [7]:
df['stress_ratings'].value_counts()

stress_ratings
medium stress    482
low stress       363
high stress      358
Name: count, dtype: int64

In [8]:
binary_lh_data = df[df['stress_ratings'].isin(['low stress', 'high stress'])]

In [9]:
X = binary_lh_data.drop(columns=['stress_ratings', 'uid', 'date'])
y = binary_lh_data['stress_ratings']
groups = binary_lh_data['uid']

stress_map = {'low stress': 0, 'high stress': 1}
y_encoded = y.map(stress_map).values 

In [10]:
logo = LeaveOneGroupOut()

best_thresholds = []
balanced_accs = []
auc_scores = []

In [11]:
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [12]:
num_splits = len(np.unique(groups))
logo = LeaveOneGroupOut()

In [13]:
num_splits = len(np.unique(groups))
class_weights = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weight_dict = dict(enumerate(class_weights))

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
    'class_weight': [class_weight_dict]
}

best_thresholds = []
balanced_accs = []
auc_scores = []
auprc_scores = []
best_params_list = []

In [14]:
with tqdm(total=num_splits, desc="LOSO CV Progress", unit="fold", bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {percentage:3.0f}%") as pbar:
    for train_idx, test_idx in logo.split(X, y_encoded, groups=groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]  

        print(f"Train shape: {X_train.shape}, {y_train.shape}")

        scaler = StandardScaler()
        X_train_normalized = scaler.fit_transform(X_train)
        X_test_normalized = scaler.transform(X_test)

        print(f"Normalized shapes: Train {X_train_normalized.shape}, Test {X_test_normalized.shape}")

        print('Performing Grid Search for SVM')
        grid_search = GridSearchCV(
            SVC(kernel='rbf', probability=True),
            param_grid,
            cv=5,
            scoring='balanced_accuracy',
            n_jobs=-1 
        )

        grid_search.fit(X_train_normalized, y_train)
        model_svm_rbf = grid_search.best_estimator_
        
        print("Best parameters:", grid_search.best_params_)
        print("Best cross-validation score:", grid_search.best_score_)
        best_params_list.append(grid_search.best_params_)

        y_test_pred_proba = model_svm_rbf.predict_proba(X_test_normalized)[:, 1]  
        thresholds = np.arange(0.01, 1.0, 0.01)
        best_threshold = max(thresholds, key=lambda t: balanced_accuracy_score(y_test, (y_test_pred_proba > t).astype(int)))

        if len(np.unique(y_test)) > 1:
            auc_score = roc_auc_score(y_test, y_test_pred_proba)
            auprc = average_precision_score(y_test, y_test_pred_proba)
            auc_scores.append(auc_score)
            auprc_scores.append(auprc)
            print(f"AUC Score: {auc_score}")
            print(f"AUPRC Score: {auprc}")
        else:
            auc_scores.append(None)
            auprc_scores.append(None)
            print(f"Skipping AUC and AUPRC computation for this fold as y_test contains only one class: {np.unique(y_test)}")

        y_test_pred_binary = (y_test_pred_proba > best_threshold).astype(int)
        balanced_acc = balanced_accuracy_score(y_test, y_test_pred_binary)
        balanced_accs.append(balanced_acc)
        best_thresholds.append(best_threshold)
        
        print(f"Balanced Accuracy: {balanced_acc}")
        print(f"Best Threshold: {best_threshold}")

        pbar.update(1)

LOSO CV Progress:   0%|                                    | 0/46 [00:00<?]   0%

Train shape: (696, 83), (696,)
Normalized shapes: Train (696, 83), Test (25, 83)
Performing Grid Search for SVM


LOSO CV Progress:   2%|▋                               | 1/46 [00:03<02:16]   2%

Best parameters: {'C': 100, 'class_weight': {0: 0.9931129476584022, 1: 1.0069832402234637}, 'gamma': 0.001}
Best cross-validation score: 0.5544143119587088
AUC Score: 0.5416666666666667
AUPRC Score: 0.6228268840920663
Balanced Accuracy: 0.5769230769230769
Best Threshold: 0.6
Train shape: (707, 83), (707,)
Normalized shapes: Train (707, 83), Test (14, 83)
Performing Grid Search for SVM


LOSO CV Progress:   4%|█▎                              | 2/46 [00:05<01:51]   4%

...

Best parameters: {'C': 10, 'class_weight': {0: 0.9931129476584022, 1: 1.0069832402234637}, 'gamma': 0.001}
Best cross-validation score: 0.5713898694811667
AUC Score: 0.3445378151260504
AUPRC Score: 0.6700922183302441
Balanced Accuracy: 0.542016806722689
Best Threshold: 0.48
Train shape: (695, 83), (695,)
Normalized shapes: Train (695, 83), Test (26, 83)
Performing Grid Search for SVM


LOSO CV Progress:  98%|█████████████████████████████▎ | 45/46 [01:41<00:02]  98%

Best parameters: {'C': 100, 'class_weight': {0: 0.9931129476584022, 1: 1.0069832402234637}, 'gamma': 0.001}
Best cross-validation score: 0.5288644975893798
AUC Score: 0.475
AUPRC Score: 0.7874928707642398
Balanced Accuracy: 0.5666666666666667
Best Threshold: 0.48
Train shape: (667, 83), (667,)
Normalized shapes: Train (667, 83), Test (54, 83)
Performing Grid Search for SVM


LOSO CV Progress: 100%|██████████████████████████████ | 46/46 [01:43<00:00] 100%

Best parameters: {'C': 10, 'class_weight': {0: 0.9931129476584022, 1: 1.0069832402234637}, 'gamma': 0.001}
Best cross-validation score: 0.5808546842650103
AUC Score: 0.5095137420718816
AUPRC Score: 0.21794307028911133
Balanced Accuracy: 0.5486257928118393
Best Threshold: 0.52


In [15]:
print('SVM with Radial basis function (RBF) kernel')
print('Median, 25th Percentile, 75th Percentile: ')

if auc_scores:
    auc_scores_valid = [score for score in auc_scores if score is not None] 

    auc_median = np.median(auc_scores_valid)
    auc_25_percentile = np.percentile(auc_scores_valid, 25)
    auc_75_percentile = np.percentile(auc_scores_valid, 75)
    print(f"AUC Score - Median: {auc_median}, 25th Percentile: {auc_25_percentile}, 75th Percentile: {auc_75_percentile}")

if auprc_scores:
    auprc_scores_valid = [score for score in auprc_scores if score is not None] 

    auprc_median = np.median(auprc_scores_valid)
    auprc_25_percentile = np.percentile(auprc_scores_valid, 25)
    auprc_75_percentile = np.percentile(auprc_scores_valid, 75)
    print(f"AUPRC Score - Median: {auprc_median}, 25th Percentile: {auprc_25_percentile}, 75th Percentile: {auprc_75_percentile}")

balanced_acc_median = np.median(balanced_accs)
balanced_acc_25_percentile = np.percentile(balanced_accs, 25)
balanced_acc_75_percentile = np.percentile(balanced_accs, 75)

print(f"Balanced Accuracy - Median: {balanced_acc_median}, 25th Percentile: {balanced_acc_25_percentile}, 75th Percentile: {balanced_acc_75_percentile}")

SVM with Radial basis function (RBF) kernel
Median, 25th Percentile, 75th Percentile: 
AUC Score - Median: 0.5095137420718816, 25th Percentile: 0.43434343434343436, 75th Percentile: 0.640625
AUPRC Score - Median: 0.6440940113065673, 25th Percentile: 0.49916457811194653, 75th Percentile: 0.7565422448065638
Balanced Accuracy - Median: 0.6666666666666666, 25th Percentile: 0.5635416666666666, 75th Percentile: 0.746875
